<a href="https://colab.research.google.com/github/mariyajoseph2002/ml/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tweet_emotions.csv to tweet_emotions.csv


In [ ]:
df=pd.read_csv('tweet_emotions.csv')

In [ ]:
print(df.head())

     tweet_id   sentiment                                            content
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696     sadness                Funeral ceremony...gloomy friday...
3  1956967789  enthusiasm               wants to hang out with friends SOON!
4  1956968416     neutral  @dannycastillo We want to trade with someone w...


In [ ]:
texts = df['content'].astype(str).tolist()  # Convert to list
labels = df['sentiment'].astype(str).tolist()


In [ ]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # Convert emotions to numbers
num_classes = len(label_encoder.classes_)

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")  # Keep top 10,000 words
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
max_length = 100  # Set max sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Multi-class classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 71s 65ms/step - accuracy: 0.2097 - loss: 2.1781 - val_accuracy: 0.2175 - val_loss: 2.1511
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 81s 64ms/step - accuracy: 0.2132 - loss: 2.1569 - val_accuracy: 0.2083 - val_loss: 2.1522
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 83s 65ms/step - accuracy: 0.2123 - loss: 2.1511 - val_accuracy: 0.2083 - val_loss: 2.1517
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 64s 64ms/step - accuracy: 0.2129 - loss: 2.1507 - val_accuracy: 0.2175 - val_loss: 2.1484
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - accuracy: 0.2151 - loss: 2.1472 - val_accuracy: 0.2083 - val_loss: 2.1498


In [ ]:
model.save("lstm_emotion_model.h5")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)
print("Tokenizer and Label Encoder saved!")

Tokenizer and Label Encoder saved!


In [ ]:
model = tf.keras.models.load_model("lstm_emotion_model.h5")

In [ ]:
with open("tokenizer.pkl", "rb") as handle:
    tokenizer = pickle.load(handle)

In [ ]:
emotion_recommendations = {
    "happiness": "Keep enjoying the moment! Try sharing your happiness with others.",
    "sadness": "Watch a favorite movie, talk to a friend, or go for a walk in nature.",
    "anger": "Take deep breaths, listen to calming music, or try meditation.",
    "fearful": "Write down your thoughts, practice grounding techniques, or talk to someone you trust.",
    "surprise": "Embrace new experiences and express your feelings through journaling.",
    "neutral": "Maintain a balanced state by engaging in hobbies and self-care activities.",
    "worry": "Take a deep breath and write down your thoughts. Focus on what you can control.",
    "fun": "Enjoy the moment! Try something new or share the joy with friends.",
    "hate": "Take a step back and breathe. Try to understand your feelings and focus on positivity.",
    "love": "Cherish the feeling! Express your love and appreciation to those who matter.",
    "enthusiasm": "Embrace your energy! Channel it into something creative or inspiring.",
    "boredom": "Try something new! Read, create, or explore a new hobby.",
    "relief": "Enjoy the calm! Take a moment to breathe and appreciate the peace.",
    "empty": "Take care of yourself. Engage in something meaningful or connect with a loved one."
}

In [ ]:
def predict_emotion(text):
    # Tokenize and pad input text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=100)

    # Predict emotion
    prediction = model.predict(padded_sequence)
    predicted_index = np.argmax(prediction)

    # Get predicted emotion
    predicted_emotion = list(emotion_recommendations.keys())[predicted_index]

    # Get recommendation
    recommendation = emotion_recommendations[predicted_emotion]

    return predicted_emotion, recommendation

In [ ]:
if __name__ == "__main__":
    user_text = input("Enter a sentence: ")
    emotion, recommendation = predict_emotion(user_text)
    print(f"Predicted Emotion: {emotion}")
    print(f"Recommendation: {recommendation}")

Enter a sentence: im not able to sleep
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted Emotion: relief
Recommendation: Enjoy the calm! Take a moment to breathe and appreciate the peace.
